In [ ]:
#############################################################################################################
# Audio-driven upper-body motion synthesis on a humanoid robot
# Computer Science Tripos Part III Project
# Jan Ondras (jo356@cam.ac.uk), Trinity College, University of Cambridge
# 2017/18
#############################################################################################################
# Automatically record synthesis on virtual robot. 
# Requires the program SimpleScreenRecorder and the pyautogui library. 
#############################################################################################################

In [ ]:
#######################################################################################################
# OFFLINE case: predictions are precomputed (loaded from file)
# (A) from the original test set
# (B) from TTS system (synthetic speech)
#######################################################################################################
# BEFORE STARTING RUN: 
# export PYTHONPATH=${PYTHONPATH}:~/Desktop/pynaoqi-python2.7-2.5.5.5-linux64/lib/python2.7/site-packages

In [ ]:
#######################################################################################################
# (A) Predictions made on NATURAL SPEECH from original test set
#######################################################################################################

import pyautogui
from naoqi import ALProxy
import numpy as np
import time

PA_pause = 0.2 # pyautogui.PAUSE not after starting the recorder

TE_folder = 'TrainingExamples_16kHz'
unique_srt_VIDs = unique_srt_VIDs = np.load('./../Dataset/'+TE_folder+'/te_unique_srt_VIDs.npz')['unique_srt_VIDs'] # sorted VIDs
all_srt_VIDs = np.load('./../Dataset/'+TE_folder+'/te_VIDs.npz')['VIDs']
unique_srt_SIDs = np.array([x[:5] for i, x in enumerate(unique_srt_VIDs) if i % 2 == 0]) # ['PID02', 'PID05', ..

model_types = [
    'MLP_SD',
    'LSTM_SD',
    'MLP_SI',
    'LSTM_SI'
]

FR = 100. # frame rate of pose features
dt = 1./FR
angles_names = [
    "HeadPitch", "HeadYaw", 
    "LShoulderRoll", "LShoulderPitch", "LElbowRoll", "LElbowYaw",
    "RShoulderRoll", "RShoulderPitch", "RElbowRoll", "RElbowYaw", 
    "HipRoll"
]
N_targets = 11
angles_used_i = np.arange(N_targets)

# Connect to the robot
IP = "127.0.0.1"
port = 38613
motionProxy = ALProxy("ALMotion", IP, port)
# print "Simulating on virtual robot ... (pose features extracted using LFTD)"


# Iterate over all subjects
for s, SID in enumerate(unique_srt_SIDs):

    # Iterate over all videos of the subject
    for v, VID in enumerate([SID + 'Task2', SID + 'Task3']):

        # Iterate over all 4 models
        for model_type in model_types:
            recording_name = model_type + "_" + VID
            print recording_name
                       
            # Load predictions
            if model_type == 'MLP_SI' or model_type == 'LSTM_SI': 
                predictions_path = './../Dataset/'+TE_folder+'/Results/' + model_type + '/cvTest/test_' + SID + '.npz'
            elif model_type == 'MLP_SD':
                predictions_path = './../Dataset/'+TE_folder+'/Results/' + model_type + '/XXXMSBMtest_' + SID + '_1_35_AF26.npz'
            elif model_type == 'LSTM_SD':
                predictions_path = './../Dataset/'+TE_folder+'/Results/' + model_type + '/MSBMtest_1_26_12_' + SID + '.npz'
            else:
                raise ValueError("Unknown model type!")
            dd = np.load(predictions_path)
            if dd['test_VIDs'][v] != VID:
                raise ValueError('Check VIDs!')
            Y_pred = dd['Y_smooth_list'][v]
            # Save the number of frames predicted by SD models; to be used to offset from the end of predictions by SI models
            if model_type == 'MLP_SD':
                LAST_N_PRED = len(Y_pred)
            # Keep last LAST_N_PRED predictions
            Y_pred = Y_pred[-LAST_N_PRED:]
            print len(Y_pred)
            
            # Reset robot to neutral pose
            for an in angles_names:
                angle_reset = 0.
                if an == 'LShoulderPitch' or an == 'RShoulderPitch':
                    angle_reset = angle_reset + np.pi/2
                motionProxy.setAngles(an, angle_reset, 1.)
            
            # Set recording name & press continue
            pyautogui.click(x=1128, y=168, pause=PA_pause)
            pyautogui.hotkey('ctrl', 'a', pause=PA_pause)
            pyautogui.press('delete', pause=PA_pause)
            pyautogui.typewrite('/home/janciovec/Desktop/RR_videos/' + recording_name + '.mp4', interval=0.05) # enter recording name
            pyautogui.click(x=1087, y=656, pause=PA_pause) # click continue
            
            # START RECORDING
            pyautogui.click(x=998, y=99)  # start
            
            # START SIMULATION
            # Simulate/synthesis
            st = time.time()
            for frame_i, angles_vector in enumerate(Y_pred):
                for angle_i in angles_used_i:
                    motionProxy.setAngles(angles_names[angle_i], angles_vector[angle_i], 1.)
                adaptive_dt = ((frame_i+1.) * dt - time.time() + st)
                #print adaptive_dt
                if adaptive_dt > 0.:
                    time.sleep(0.95*adaptive_dt)
            
            # STOP RECORDING & click "Save recording" & prepare for next one
            pyautogui.click(x=998, y=99, pause=PA_pause)   # stop
            pyautogui.click(x=1087, y=656, pause=PA_pause) # save
            pyautogui.click(x=1087, y=656, pause=PA_pause) # back to start
            pyautogui.click(x=1087, y=656, pause=PA_pause) # click continue
            pyautogui.click(x=1087, y=656, pause=PA_pause) # click continue
            
            et = time.time()
            print "\tTotal simulation time: ", et - st, " s = ", (et - st)/60., " min"
            print "===================================================================================\n"
            time.sleep(3.)

In [ ]:
#######################################################################################################
# (B) Predictions made on SYNTHETIC SPEECH
#######################################################################################################

import pyautogui
from naoqi import ALProxy
import numpy as np
import time

PA_pause = 0.2 # pyautogui.PAUSE, not after starting the recorder

TE_folder = 'TrainingExamples_16kHz'

FR = 100. # frame rate of pose features
dt = 1./FR
angles_names = [
    "HeadPitch", "HeadYaw", 
    "LShoulderRoll", "LShoulderPitch", "LElbowRoll", "LElbowYaw",
    "RShoulderRoll", "RShoulderPitch", "RElbowRoll", "RElbowYaw", 
    "HipRoll"
]
N_targets = 11
angles_used_i = np.arange(N_targets)

# Connect to the robot
IP = "127.0.0.1"
port = 45699
motionProxy = ALProxy("ALMotion", IP, port)
# print "Simulating on virtual robot ... (pose features extracted using LFTD)"

TTS_methods = [
    'MOB', # MaryTTS, voice obadiah
    'MSP', # MaryTTS, voice spike
    'MPR', # MaryTTS, voice prudence
    'MPO'  # MaryTTS, voice poppy
]

save_results_path_prefix = './../Dataset/Synthetic_TTS/'

model_types = ['LSTM_SI', 'MLP_SI']

IDs = ['6', '7', '8', '9'] # stories Banana, Picnic, Army, Glasses: http://docs.autismresearchcentre.com/papers/1999_Jolliffe_BC_Stories.pdf

# Iterate over 2 models
for model_type in model_types:
    # Iterate over 4 stories
    for ID in IDs:
        # Iterate over 4 voices
        for TTS_method in TTS_methods:

            recording_name = model_type + "_SYNTHETIC" + ID + TTS_method
            print recording_name

            # Load predictions
            dd = np.load(save_results_path_prefix + 'SYNTHETIC_pred_' + model_type + '_' + ID + '_' + TTS_method + '.npz')
            Y_pred = dd['Y_smooth']

            # Reset robot to neutral pose
            for an in angles_names:
                angle_reset = 0.
                if an == 'LShoulderPitch' or an == 'RShoulderPitch':
                    angle_reset = angle_reset + np.pi/2
                motionProxy.setAngles(an, angle_reset, 1.)

            # Set recording name & press continue
            pyautogui.click(x=1128, y=168, pause=PA_pause)
            pyautogui.hotkey('ctrl', 'a', pause=PA_pause)
            pyautogui.press('delete', pause=PA_pause)
            pyautogui.typewrite('/home/janciovec/Desktop/RR_videos/' + recording_name + '.mp4', interval=0.05) # enter recording name
            pyautogui.click(x=1087, y=656, pause=PA_pause) # click continue

            # START RECORDING
            pyautogui.click(x=998, y=99)  # start

            # START SIMULATION
            # Simulate/synthesis
            st = time.time()
            for frame_i, angles_vector in enumerate(Y_pred):
                for angle_i in angles_used_i:
                    motionProxy.setAngles(angles_names[angle_i], angles_vector[angle_i], 1.)
                #time.sleep(dt) # delay by 1/FPS
                adaptive_dt = ((frame_i+1.) * dt - time.time() + st)
                #print adaptive_dt
                if adaptive_dt > 0.:
                    time.sleep(0.95*adaptive_dt)

            # STOP RECORDING & click "Save recording" & prepare for next one
            pyautogui.click(x=998, y=99, pause=PA_pause)   # stop
            pyautogui.click(x=1087, y=656, pause=PA_pause) # save
            pyautogui.click(x=1087, y=656, pause=PA_pause) # back to start
            pyautogui.click(x=1087, y=656, pause=PA_pause) # click continue
            pyautogui.click(x=1087, y=656, pause=PA_pause) # click continue

            et = time.time()
            print "\tTotal simulation time: ", et - st, " s = ", (et - st)/60., " min"
            print "===================================================================================\n"
            time.sleep(3.)